In [6]:
import pandas as pd 

In [7]:
def SearchMax(dataFrame) :
    Max=0
    for data in dataFrame[1]:
        if data >= Max :
            Max=data
    
    return Max

In [8]:
def SearchMin(dataFrame) :
    Min=dataFrame[1][0]
    for data in dataFrame[1]:
        if data <= Min :
            Min=data
    return Min

In [9]:
#지표계산
def IndicationCal(dataFrame) :
    mindata = SearchMin(dataFrame)
    maxdata = SearchMax(dataFrame)
    number = 0
    dataFrame = dataFrame.astype({1:float})
    for data in dataFrame[1]:
        dataFrame[1][number] = round((data-mindata)/(maxdata-mindata)*10,3)
        number += 1
    return dataFrame

In [10]:
#지표계산 역방향
def IndicationCalReverse(dataFrame) :
    mindata = SearchMin(dataFrame)
    maxdata = SearchMax(dataFrame)
    number = 0
    dataFrame = dataFrame.astype({1:float})
    for data in dataFrame[1]:
        dataFrame[1][number] = round(10-((data-mindata)/(maxdata-mindata)*10),3)
        number += 1
    return dataFrame

In [11]:
#업종에 따른 모든 구의 지표값*가중치 값 
def WeightSet(floatPopulation, competeSector, officialPrice) :
    f_p = IndicationCal(floatPopulation)
    c_s = IndicationCalReverse(competeSector)
    o_p = IndicationCalReverse(officialPrice)
    number_f = 0
    recoList = pd.DataFrame(columns = ['State' , 'Recommed'])
    for state_f in f_p[0]:
        number_c = 0
        number_o = 0
        sumdata = 0.0
        sumdata += (f_p[1][number_f]*0.3)
        for state_c in c_s[0]:
            if state_f == state_c :
                sumdata += (c_s[1][number_c]*0.4)
            number_c += 1
        for state_o in o_p[0]:
            if state_f == state_o :
                sumdata += (o_p[1][number_o]*0.3)
            number_o += 1
        recoList.loc[number_f]=[state_f, sumdata]
        number_f += 1
    return recoList

In [12]:
x = pd.DataFrame([['종로구', 12, 3], ['구로구', 5, 6], ['남로구', 8, 9], ['종구', 5, 6], ['구', 8, 9]])
y = pd.DataFrame([['구', 12, 3], ['종구', 5, 6], ['남로구', 8, 9], ['구로구', 5, 6], ['종로구', 8, 9]])
z = pd.DataFrame([['구로구', 12, 3], ['남로구', 5, 6], ['구', 5, 6], ['종로구', 8, 9], ['종구', 8, 9]])


In [13]:
k=WeightSet(x, y, z)
k

<ipython-input-9-6a6c2ca5c093>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[1][number] = round((data-mindata)/(maxdata-mindata)*10,3)


,State,Recommed
0,종로구,6.9998
1,구로구,4.0000
2,남로구,6.5714
3,종구,5.7142
4,구,4.2858
